In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from rake_nltk import Rake
from pytrends.request import TrendReq
from openpyxl import Workbook

# -------------------------
# STEP 1: SCRAPE WEBSITE
# -------------------------
url = "https://teaspoonofadventure.com/eat-in-spain"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "lxml")

# Extracting Elements
title = soup.title.text.strip() if soup.title else ""
meta_desc = soup.find("meta", attrs={"name": "description"})
meta_desc = meta_desc["content"].strip() if meta_desc else ""
h_tags = [tag.text.strip() for tag in soup.find_all(["h1", "h2", "h3"])]
body_text = " ".join([p.text for p in soup.find_all("p")])
blog_tags = [tag.text.strip() for tag in soup.find_all("a", class_="tag")]  # Adjust class if different
internal_links = [a['href'] for a in soup.find_all("a", href=True) if "teaspoonofadventure.com" in a['href']]

print("✅ Web Scraping Completed")

# -------------------------
# STEP 2: NLP KEYWORD EXTRACTION
# -------------------------
nltk.download('stopwords')
r = Rake(stopwords=stopwords.words('english'))
r.extract_keywords_from_text(body_text)
keywords_ranked = r.get_ranked_phrases()

# Long-tail (2-4 words only)
long_tail_keywords = [kw for kw in keywords_ranked if 2 <= len(kw.split()) <= 4]
long_tail_keywords = list(dict.fromkeys(long_tail_keywords))  # Remove duplicates

print(f"✅ Extracted {len(long_tail_keywords)} Long-Tail Keywords")

# -------------------------
# STEP 3: PYTRENDS ANALYSIS
# -------------------------
pytrends = TrendReq(hl='en-US', tz=360)
volume_data = []
for kw in long_tail_keywords[:300]:  # Limit to first 10 for speed
    try:
        pytrends.build_payload([kw])
        interest = pytrends.interest_over_time()
        avg_volume = interest[kw].mean() if not interest.empty else 0
        volume_data.append((kw, avg_volume))
    except:
        volume_data.append((kw, 0))

# Simulated Difficulty & CPC (as PyTrends doesn’t provide directly)
import random
keyword_data = []
for kw, vol in volume_data:
    keyword_data.append({
        "Keyword": kw,
        "Source URL": url,
        "Volume": int(vol),
        "Difficulty": random.randint(10, 90),  # Simulated
        "CPC": round(random.uniform(0.5, 5.0), 2),  # Simulated
        "Type": "Long-tail" if len(kw.split()) > 1 else "Head"
    })

df = pd.DataFrame(keyword_data)
print("✅ PyTrends Analysis Completed")

# -------------------------
# STEP 4: AUTOMATED REPORT
# -------------------------
excel_file = "Spain_Travel_Keywords.xlsx"
df.to_excel(excel_file, index=False)
print(f"✅ Report Saved as {excel_file}")


✅ Web Scraping Completed


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Extracted 258 Long-Tail Keywords


C:\Users\Hp\anaconda3\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
C:\Users\Hp\anaconda3\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
C:\Users\Hp\anaconda3\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('fut

✅ PyTrends Analysis Completed
✅ Report Saved as Spain_Travel_Keywords.xlsx


C:\Users\Hp\anaconda3\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from rake_nltk import Rake
from nltk.corpus import stopwords
from pytrends.request import TrendReq
from openpyxl import Workbook
import time

# -------------------------
# STEP 1: SCRAPE WEBSITE
# -------------------------
def scrape_seo_elements(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "lxml")

    title = soup.title.string if soup.title else ""
    meta_desc = soup.find("meta", {"name": "description"})
    meta_desc = meta_desc["content"] if meta_desc else ""

    headings = [h.get_text(strip=True) for h in soup.find_all(['h1', 'h2', 'h3'])]
    body_content = " ".join([p.get_text(strip=True) for p in soup.find_all('p')])

    # Blog Tags (Assume tags have <a rel='tag'> or 'tag' in class)
    tags = [tag.get_text(strip=True) for tag in soup.find_all("a", {"rel": "tag"})]
    if not tags:
        tags = [tag.get_text(strip=True) for tag in soup.find_all("a", class_=re.compile("tag"))]

    internal_links = [a["href"] for a in soup.find_all("a", href=True) if url.split("/")[2] in a["href"]]

    return {
        "title": title,
        "meta_desc": meta_desc,
        "headings": headings,
        "body_content": body_content,
        "tags": tags,
        "internal_links": internal_links
    }

# -------------------------
# STEP 2: KEYWORD EXTRACTION
# -------------------------
def extract_keywords(text, min_keywords=300):
    r = Rake(stopwords=stopwords.words('english'))
    r.extract_keywords_from_text(text)
    phrases = r.get_ranked_phrases()

    # Ensure at least 300 keywords (repeat or trim)
    if len(phrases) < min_keywords:
        phrases = phrases * (min_keywords // len(phrases) + 1)
    phrases = list(dict.fromkeys(phrases))[:min_keywords]  # remove duplicates & trim

    # Classify into short, mid, long tail
    short_tail, mid_tail, long_tail = [], [], []
    for kw in phrases:
        wc = len(kw.split())
        if wc == 2:
            short_tail.append(kw)
        elif wc == 3:
            mid_tail.append(kw)
        elif wc >= 4:
            long_tail.append(kw)

    return short_tail, mid_tail, long_tail

# -------------------------
# STEP 3: PYTRENDS KEYWORD ANALYSIS
# -------------------------
def analyze_keywords_pytrends(keywords, max_keywords=50):
    pytrends = TrendReq(hl='en-US', tz=330)
    data = []

    for kw in keywords[:max_keywords]:  # PyTrends works best with <50 per batch
        try:
            pytrends.build_payload([kw], timeframe='today 12-m', geo='IN')
            time.sleep(1)  # avoid rate limit

            df_trend = pytrends.interest_over_time()
            volume = df_trend[kw].mean() if not df_trend.empty else 0
            difficulty = round((100 - volume) / 10, 2)  # Fake metric (proxy)
            cpc = round(volume / 10, 2)  # Fake CPC proxy (for demonstration)
            data.append([kw, volume, difficulty, cpc])
        except:
            data.append([kw, 0, 0, 0])
            continue
    return data

# -------------------------
# STEP 4: FINAL REPORT
# -------------------------
def create_final_report(url, short_tail, mid_tail, long_tail):
    all_keywords = (
        [(kw, "Short-tail") for kw in short_tail] +
        [(kw, "Mid-tail") for kw in mid_tail] +
        [(kw, "Long-tail") for kw in long_tail]
    )
    keywords_list = [kw[0] for kw in all_keywords]
    trends_data = analyze_keywords_pytrends(keywords_list)

    final_data = []
    for (kw, tail_type), (_, volume, difficulty, cpc) in zip(all_keywords, trends_data):
        final_data.append([kw, url, volume, difficulty, cpc, tail_type])

    df = pd.DataFrame(final_data, columns=["Keyword", "Source_URL", "Volume", "Difficulty", "CPC", "Tail-Type"])
    df.to_excel("SEO_Report.xlsx", index=False)
    print("\n✅ Report saved as SEO_Report.xlsx")
    print(df.head(10))


# -------------------------
# STEP 5: RUNNING ALL STEPS
# -------------------------
if __name__ == "__main__":
    url = "https://teaspoonofadventure.com/eat-in-spain/"
    print(f"Scraping SEO data from: {url} ...")
    seo_data = scrape_seo_elements(url)
    print("Title:", seo_data['title'])
    print("Meta Description:", seo_data['meta_desc'])
    print("Headings Found:", seo_data['headings'][:5])
    print("Tags:", seo_data['tags'])

    print("\nExtracting keywords...")
    text_to_analyze = " ".join([seo_data['title'], seo_data['meta_desc']] + seo_data['headings']) + " " + seo_data['body_content']
    short_tail, mid_tail, long_tail = extract_keywords(text_to_analyze)

    print(f"Short-tail: {len(short_tail)}, Mid-tail: {len(mid_tail)}, Long-tail: {len(long_tail)}")
    create_final_report(url, short_tail, mid_tail, long_tail)


Scraping SEO data from: https://teaspoonofadventure.com/eat-in-spain/ ...
Title: 11 Things You Must Eat in Spain
Meta Description: It's not just paella and sangria! Here are 11 things you must eat in Spain, from burgers and wine to seafood and lots of dessert!
Headings Found: ['11 Things You Must Eat in Spain', 'Must eat in Spain: Fideuà', 'What to eat in Spain? Spanish seafood, of course!', 'Don’t miss paella on a trip to Spain', 'Who knew they had amazing burgers in Barcelona?']
Tags: ['Europe', 'Spain', 'Travel']

Extracting keywords...
Short-tail: 194, Mid-tail: 66, Long-tail: 28


C:\Users\Hp\anaconda3\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
C:\Users\Hp\anaconda3\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
C:\Users\Hp\anaconda3\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('fut


✅ Report saved as SEO_Report.xlsx
          Keyword                                     Source_URL     Volume  \
0         rest ..  https://teaspoonofadventure.com/eat-in-spain/   4.296296   
1  dining culture  https://teaspoonofadventure.com/eat-in-spain/   2.111111   
2         go back  https://teaspoonofadventure.com/eat-in-spain/  62.796296   
3          post 🙂  https://teaspoonofadventure.com/eat-in-spain/   0.000000   
4       new place  https://teaspoonofadventure.com/eat-in-spain/  48.481481   
5  dessert person  https://teaspoonofadventure.com/eat-in-spain/   2.259259   
6  patatas bravas  https://teaspoonofadventure.com/eat-in-spain/   5.444444   
7   great options  https://teaspoonofadventure.com/eat-in-spain/  44.500000   
8       san gines  https://teaspoonofadventure.com/eat-in-spain/   0.000000   
9     good paella  https://teaspoonofadventure.com/eat-in-spain/   0.000000   

   Difficulty   CPC   Tail-Type  
0        9.57  0.43  Short-tail  
1        9.79  0.21  Short-

C:\Users\Hp\anaconda3\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
